In [1]:
from crossref.restful import Works
from crossref.restful import Journals
from time import sleep
import numpy as np
from bs4 import BeautifulSoup, NavigableString
import csv
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import random
import math

### Read titles

In [3]:
df=pd.read_csv('JMR2000-2022.csv')
len(df["Title"].unique())

1265

### Collect citations

In [49]:
titles=df['Title'][1000:]
## 630-760 is good
# url_lst=[]
# for title in titles:
#     url = make_url_gufen(title)
#     url_lst.append(url)
# url_lst.insert(0, 'https://xs.studiodahu.com/scholar?q=TEST')
titles
    

1000    Open Negotiation: The Back-End Benefits of Sal...
1001    Opening the Umbrella: The Effects of Rebrandin...
1002                      Opportunity Cost Overestimation
1003    Optimal Design of Free Samples for Digital Pro...
1004    P2V-MAP: Mapping Market Structures for Large R...
                              ...                        
1276    We Succeeded Together, Now What: Relationship ...
1277    When and How Should Firms Differentiate? Quali...
1278    Women and Minority Film Directors in Hollywood...
1279    You Must Have a Preference: The Impact of No-P...
1280    You Should Try These Together: Combinatory Rec...
Name: Title, Length: 281, dtype: object

In [249]:
# final_titles = []
# for i in range(len(df_check)):
#     if math.isnan(df_check['Citation'][i]) == True:
#         final_titles.append(df_check['Title'][i])
# len(final_titles)

172

In [50]:
#For ChromeDriver version 79.0.3945.16 or over
options = Options()
options.add_argument('--disable-blink-features=AutomationControlled') 

#Open Browser
driver = webdriver.Chrome(executable_path='/Users/richardpang/.wdm/drivers/chromedriver/mac64/106.0.5249/chromedriver',options=options)
citation_lst = []
count = 0
for title in titles:
# driver = webdriver.Chrome( '/Users/richardpang/.wdm/drivers/chromedriver/mac64/106.0.5249/chromedriver')
    a = f"{'+'.join(title.split())}"
    url = 'https://xs.studiodahu.com/scholar?q=' + a 
    driver.get(url)
    element = driver.find_elements(By.TAG_NAME, 'div')
    element = driver.find_elements(By.CLASS_NAME, 'gs_fl')
    # print(len(element))
    if len(element)>0:
        # element.select_one(":nth-child(2)")
        words = element[1].text.split()
        # print(words)
        for word in words:
            if word.isdigit():
                citation_lst.append([title,word])
                break
        # citation = element[1].text.split('Cited by ')[1].split()[0]
        # print([titles[i-1],citation])
    # if len(element)>4:
    #     title = element[0].text.split('\n')[3]
    #     abstract = element[2].text.split('\n')[1]
    #     main_text = element[3].text
    #     doi = doi_lst[i]
    #     article  = [title,abstract,main_text,doi]
    #     article_lst.append(article)
    count += 1
    if count%10 ==0:
        print(count, 'articles have been collected.')
    sleep(2)
driver.quit()
citation_df = pd.DataFrame(citation_lst,columns=['Title','Citation'])
citation_df

/var/folders/28/rtg76x2d1m791yxskrhmnffc0000gn/T/ipykernel_98347/1651243265.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/richardpang/.wdm/drivers/chromedriver/mac64/106.0.5249/chromedriver',options=options)


10 articles have been collected.
20 articles have been collected.
30 articles have been collected.
40 articles have been collected.
50 articles have been collected.
60 articles have been collected.
70 articles have been collected.
80 articles have been collected.
90 articles have been collected.
100 articles have been collected.
110 articles have been collected.
120 articles have been collected.
130 articles have been collected.
140 articles have been collected.
150 articles have been collected.
160 articles have been collected.
170 articles have been collected.
180 articles have been collected.
190 articles have been collected.
200 articles have been collected.
210 articles have been collected.
220 articles have been collected.
230 articles have been collected.
240 articles have been collected.
250 articles have been collected.
260 articles have been collected.
270 articles have been collected.
280 articles have been collected.


,Title,Citation
0,Opening the Umbrella: The Effects of Rebrandin...,25
1,Opportunity Cost Overestimation,12
2,Optimal Design of Free Samples for Digital Pro...,55
3,P2V-MAP: Mapping Market Structures for Large R...,38
4,Personalizing the Customization Experience: A ...,40
...,...,...
264,Understanding Lateral and Vertical Biases in C...,11
265,Variety Effects in Mobile Advertising,3
266,"We Succeeded Together, Now What: Relationship ...",2
267,When and How Should Firms Differentiate? Quali...,4


In [51]:
# citation_df = pd.DataFrame(citation_lst,columns=['Title','Citation'])
citation_df['Title'][8]

'Positioning Multicountry Brands: The Impact of Variation in Cultural Values and Competitive Set'

### Original data

In [52]:
df = pd.read_csv('JMR2000-2022_with_citation.csv')

In [14]:
# df["Citation"] = np.nan

#### Count how many citations are now available

In [53]:
count = 0
for i in df['Citation']:
    # print(i,type(i))
    if math.isnan(i) == False:
        count += 1
count

985

#### Update citations

In [54]:
for j in range(len(citation_df)):
    title_citation_df = citation_df['Title'][j]
    citation_citation_df = citation_df['Citation'][j]

    for i in range(len(df)):
        title_df = df['Title'][i]
        if title_citation_df == title_df:
            # print(citation_cit)
            df['Citation'][i] = float(citation_citation_df)


/var/folders/28/rtg76x2d1m791yxskrhmnffc0000gn/T/ipykernel_98347/3561171303.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Citation'][i] = float(citation_citation_df)


### Check citations after updates, it should increase

In [55]:
count=0
for i in df['Citation']:
    # print(i,type(i))
    if math.isnan(i) == False:
        count += 1
count

1254

### Save the update citations

In [56]:
df.to_csv('JMR2000-2022_with_citation.csv')

In [57]:
df_check = pd.read_csv('JMR2000-2022_with_citation.csv')
count=0
for i in df_check['Citation']:
    # print(i,type(i))
    if math.isnan(i) == False:
        count += 1
count

1254

In [266]:
final_titles = []
result = [2438,0,0,0,0,295,0,69,237]
j = 0
for i in range(len(df_check)):
    if math.isnan(df_check['Citation'][i]) == True:
        df_check['Citation'][i] = result[j]
        j += 1


/var/folders/28/rtg76x2d1m791yxskrhmnffc0000gn/T/ipykernel_60958/4116885861.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_check['Citation'][i] = result[j]


In [268]:
count = 0
for i in range(len(df_check)):
    if math.isnan(df_check['Citation'][i]) == True:
        count += 1
count

0

In [272]:
df= pd.read_csv('/Users/richardpang/Desktop/extract_maintext-1/JM2000-2022_citation.csv')

AttributeError: 'DataFrame' object has no attribute 'unique'